In [8]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
#import cv2
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
os.environ['KERAS_BACKEND'] = "plaidml.keras.backend"
import keras
import pandas as pd

from os import listdir
from os.path import isdir,isfile,join
import shutil

import tensorflow_hub as hub

Using plaidml.keras.backend backend.


In [39]:
path = 'C:/Users/norma/Downloads/data' #'C:/Users/norma/Downloads/VSCNN/data'

In [40]:
training_files=pd.read_csv(path+'/user/train.txt', sep= ' ', header=None, names=['file', 'class'])
training_files= training_files[training_files['class'] == 0]['file'].str.replace(r'train/', '').to_list()
#testing_files['first'] = testing_files['class'].str[:1]

In [42]:
matches = training_files
for filename in os.listdir(path+'/user/train'):
    if filename in matches:
        shutil.copy(path+'/user/train/'+filename,path+'/Us/train/0')
    else:
        shutil.copy(path+'/user/train/'+filename,path+'/Us/train/1')

In [45]:
validation_Boston =pd.read_csv(path+'/user/val_Boston.txt', sep= ' ', header=None, names=['file', 'class'])
validation_Boston['class'] = validation_Boston['class'].astype('category')
validation_Boston = validation_Boston[validation_Boston['class'] == 0]['file'].str.replace(r'val/Boston/', '').to_list()
#validation_Boston['first'] = validation_Boston['class'].str[:1]
 

In [46]:
for filename in os.listdir(path+'/user/val/Boston'): 
    if filename in validation_Boston:
        shutil.copy(path+'/user/val/Boston/'+filename,path+'/Us/val/0')
    else:
        shutil.copy(path+'/user/val/Boston/'+filename,path+'/Us/val/1')

In [30]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = path+'/train'
validation_dir = path+'/val'

BATCH_SIZE = 32
IMG_SIZE = (227, 227)

train_dataset = image_dataset_from_directory(train_dir[:10000],
                                             shuffle=True, 
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                             shuffle=True, 
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 72950 files belonging to 2 classes.
Found 2048 files belonging to 2 classes.


In [31]:
class_names = train_dataset.class_names

In [33]:
from tensorflow.keras import layers
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [34]:
normalized_train = train_dataset.map(lambda x, y: (normalization_layer(x), y))
normalized_val = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

#image_batch, labels_batch = next(iter(normalized_ds))

In [35]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = normalized_train.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = normalized_val.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 227, 227, 3)
(32,)


In [36]:
model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(2)
])

In [37]:
model.compile(
  optimizer='adam',
  loss=tf.losses.BinaryCrossentropy(),
  metrics=['accuracy'])

In [38]:
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=3
         )
          

Epoch 1/3
2280/2280 [==============================] - 1355s 594ms/step - loss: 4.2060 - accuracy: 0.5000 - val_loss: 4.2042 - val_accuracy: 0.5000
Epoch 2/3
 125/2280 [>.............................] - ETA: 25:03 - loss: 4.1672 - accuracy: 0.4952

KeyboardInterrupt: 

In [17]:
feature_extractor_model = "https://tfhub.dev/google/imagenet/inception_v3/classification/4"

In [18]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_model, input_shape=(227, 227, 3), trainable=False)

In [19]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

(32, 1001)


In [20]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(2)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              23853833  
_________________________________________________________________
dense (Dense)                (None, 2)                 2004      
Total params: 23,855,837
Trainable params: 2,004
Non-trainable params: 23,853,833
_________________________________________________________________


In [21]:
predictions = model(image_batch)

In [22]:
predictions.shape

TensorShape([32, 2])

In [27]:
model.compile(
  optimizer='adam',
  loss=tf.losses.BinaryCrossentropy(),
  metrics=['accuracy'])

In [28]:
model.fit(train_dataset,
          validation_data=validation_dataset,
          epochs=3
         )

Epoch 1/3


ValueError: in user code:

    C:\Users\norma\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\norma\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\norma\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\norma\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\norma\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\norma

In [ ]:
Business_train = []
for n in range(0, len(os.listdir('C:/Users/norma/Downloads/VSCNN/data/business/train')[:10000])):
    Business_train.append(cv2.imread(path+'/business/train/'+os.listdir('C:/Users/norma/Downloads/VSCNN/data/business/train')[n]))
    print(n)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale = 1./255)
batch_size = 32
img_width, img_height = 227, 227
train_data_dir = path+'/business/train'

In [ ]:
data_train = train_generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

X_tra = []
for i in range(0, len(Business_train)):
    X_tra.append(cv2.resize(Business_train[i], (227, 227), interpolation=cv2.INTER_AREA))

In [ ]:
len(os.listdir('C:/Users/norma/Downloads/VSCNN/data/business/train')[:10000])

In [ ]:
X_train = []
for i in range(len(X_tra)):
    X_train.append(X_tra[i]/255.0)

In [ ]:
y_train = pd.read_csv(path+'/business/train.txt', sep= ' ', header=None, names=['file', 'class'])
y_train = y_train['class'].to_list()

In [ ]:
plt.imshow(X_train[5000])

In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
Model_prueba = tf.keras.Sequential([
    feature_extractor_layer,
    layer.Dense(2)
])

In [ ]:
feurl = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
fel = hub.KerasLayer(feurl,
                    input_shape = [227, 227, 3])

fb = fel.build([None, 27, 227, 3])
fel.trainable = False



In [ ]:
m = tf.keras.Sequential([
    fel,
    layers.Dense(2)
    ])